`pip install llama-index-llms-openai`  
`pip install llama-index-graph-stores-neo4j`   
`pip install llama-index-embeddings-openai`  
`pip install neo4j`  
`pip install python-dotenv`  

Download neo4j desktop client using https://neo4j.com/download/ or use neo4j aura with https://neo4j.com/cloud/platform/aura-graph-database/  
Create a new project, set the password and select Plugins to install APOC to the database. 

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_KEY')

import logging
import sys
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# define LLM
llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.llm = llm
Settings.chunk_size = 512

In [2]:
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.graph_stores.neo4j import Neo4jGraphStore


from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display

In [14]:
documents = SimpleDirectoryReader(
    input_files=["data/harry_potter/1.txt", "data/harry_potter/2.txt", "data/harry_potter/3.txt", "data/harry_potter/4.txt", "data/harry_potter/5.txt"]
).load_data()

In [15]:
username = "neo4j"
password = os.getenv('NEO4J_PW')
url = os.getenv('NEO4J_URL')
database = "neo4j"

In [16]:
graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database=database,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)

# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=2,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

In [42]:
from neo4j import GraphDatabase

cypher_query = """
MATCH p = (:Entity { id: "Harry" })-[*1]->()
RETURN p;
"""

with GraphDatabase.driver(uri=os.getenv('NEO4J_URL'), auth=('neo4j', os.getenv('NEO4J_PW'))) as driver:
    with driver.session() as session:
        result = session.run(cypher_query)

        # keep within context manager to ensure no consume error is thrown

        # TODO: some logic to manipulate result object 



In [11]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever

# graph_rag_retriever = KnowledgeGraphRAGRetriever(
#     storage_context=storage_context,
#     verbose=True,
# )

# query_engine = RetrieverQueryEngine.from_args(
#     graph_rag_retriever,
# )
query_engine = index.as_query_engine(
    include_text=True, response_mode="tree_summarize"
)

In [12]:
response = await query_engine.aquery(
    "Why is Harry Potter special",
)
display(Markdown(f"<b>{response}</b>"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: f4772a98-95c7-4d6b-be25-cd9cd08a4bb0: Uncle Vernon entered the kitchen as Harry was turning over the bacon.

"Comb ...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 3904a4dd-c407-4193-a626-09b73edd4258: "Young Sirius Black lent it to
me. I've got him, sir."

"No problems, were th...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 4e908b69-886d-40f1-837b-bcfa3c923d9f: "On vacation in Majorca," snapped Aunt Petunia.

"You could just leave me her...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


<b>Harry Potter is considered special because he survived an attack as a baby that left him with a lightning-shaped scar on his forehead, a scar that is said to be permanent. This event is significant because it marked him as the only known survivor of the Killing Curse, which was cast by the dark wizard Lord Voldemort. Additionally, Harry's survival and the circumstances surrounding it have led to him being known as "The Boy Who Lived" in the wizarding world.</b>